In [2]:
import numpy as np
import pickle
import random

In [3]:
filenames = pickle.load(open('data/filenames-caltech101.pickle', 'rb'))
feature_list = pickle.load(open('data/features-caltech101-mobilenet.pickle',
                                'rb'))

In [4]:
num_images = len(filenames)
num_features_per_image = len(feature_list[0])

In [5]:
random_image_index = random.randint(0, num_images)

PCA

In [6]:
num_feature_dimensions = 100
num_feature_dimensions = min(num_images, num_feature_dimensions,
                             len(feature_list[0]))

In [8]:
from sklearn.decomposition import PCA

In [9]:
pca = PCA(n_components=num_feature_dimensions)
pca.fit(feature_list)
feature_list_compressed = pca.transform(feature_list)
feature_list_compressed = feature_list_compressed.tolist()

In [16]:
from annoy import AnnoyIndex
import time

In [17]:
# Time the indexing for Annoy
t = AnnoyIndex(100)  # Length of item vector that will be indexed
starttime = time.time()
for i in range(num_images-1):
    feature = feature_list_compressed[i]
    t.add_item(i, feature)
endtime = time.time()
print(endtime - starttime)
t.build(40)  # 50 trees
t.save('data/caltech101index.ann')

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  


0.03657102584838867


True

Annoy on one image

In [18]:
u = AnnoyIndex(100)
%timeit u.get_nns_by_vector(feature_list_compressed[random_image_index], 5, include_distances=True)
indexes = u.get_nns_by_vector(feature_list_compressed[random_image_index],
                              5,
                              include_distances=True)

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  """Entry point for launching an IPython kernel.


3.95 µs ± 511 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [25]:
def calculate_annoy_time():
    for i in range(0, 6000):
        indexes = u.get_nns_by_vector(feature_list_compressed[i],
                                      5,
                                      include_distances=True)

Annoy on total images

In [26]:
%timeit calculate_annoy_time()

21.9 ms ± 451 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
